In [4]:
import requests
%pip install streamlit
import pandas as pd
import streamlit as st

%pip install altair pandas vega_datasets
%pip install -U altair_viewer
%pip install streamlit
import altair as alt

  Obtaining dependency information for streamlit from https://files.pythonhosted.org/packages/d3/96/9251b421d0a1c7d625a82a04bea56b8a9830c785940ec16db454b85c6db7/streamlit-1.29.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for blinker<2,>=1.0.0 from https://files.pythonhosted.org/packages/fa/2a/7f3714cbc6356a0efec525ce7a0613d581072ed6eb53eb7b9754f33db807/blinker-1.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for cachetools<6,>=4.0 from https://files.pythonhosted.org/packages/a2/91/2d843adb9fbd911e0da45fbf6f18ca89d07a087c3daa23e955584f90ebf4/cachetools-5.3.2-py3-none-any.whl.metadata
  Obtaining dependency information for protobuf<5,>=3.20 from https://files.pythonhosted.org/packages/fe/6b/7f177e8d6fe4caa14f4065433af9f879d4fab84f0d17dcba7b407f6bd808/protobuf-4.25.1-cp310-abi3-win_amd64.whl.metadata
  Obtaining dependency information for rich<14,>=10.14.0 from https://files.pythonhosted.org/packages/be/be/1520178fa01eabe014b16e72a952b9f900631142ccd

In [13]:
jsonlist = requests.get("https://api.census.gov/data/2019/pep/charagegroups?get=NAME,POP&HISP=2&for=state:*").json()
states =[]

populations =[]
for list in jsonlist:
    states.append(list[0])
    populations.append(list[1])
data = {"State":states,"Population":populations
}
df=pd.DataFrame(data)
df = df.iloc[1:]
df["Population"]=pd.to_numeric(df["Population"])
df.head()




,State,Population
1,Mississippi,100110
2,Missouri,268708
3,Montana,43289
4,Nebraska,219645
5,Nevada,900600


In [14]:
chart = alt.Chart(df).mark_area().encode(
    x="State",
    y="Population"
)
st.title('Population Chart')
st.altair_chart(
    chart, use_container_width=True)


c:\Users\alexf\anaconda3\Lib\site-packages\pyarrow\pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


DeltaGenerator()

Vega-altair orders state IDs in aphabetical order so I ordered the data frame by state name

In [15]:
from vega_datasets import data
df = df.sort_values(by="State")
df=df.reset_index(drop=True)
#df['id']= data.population_engineers_hurricanes()
df.head()
## just a random vega dataset that uses same id convention for states
altair_state_ids = data.population_engineers_hurricanes()
df["id"]=altair_state_ids['id']
df.head()


,State,Population,id
0,Alabama,223278,1
1,Alaska,53212,2
2,Arizona,2310590,4
3,Arkansas,236631,5
4,California,15574880,6


In [16]:
from vega_datasets import data

states = alt.topo_feature(data.us_10m.url, 'states')


alt.Chart(states).mark_geoshape().encode(
    color='Population:Q',
    tooltip=['State:N', 'Population:Q']
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(df, 'id', ['Population','State'])
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)


alt.Chart(...)

In [17]:
import altair as alt
from vega_datasets import data

# Load US states TopoJSON data
states = alt.topo_feature(data.us_10m.url, 'states')
# Display the first feature's properties
alt.Chart(states).mark_geoshape().transform_calculate(
    properties=alt.datum.properties
).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)


alt.Chart(...)

List of state names and state codes in corresponding order

In [18]:
state_codes = [
    "AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DE", "FL", "GA",
    "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD",
    "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ",
    "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC",
    "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"
]

state_names = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California",
    "Colorado", "Connecticut", "Delaware", "Florida", "Georgia",
    "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas",
    "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts",
    "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana",
    "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico",
    "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma",
    "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota",
    "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington",
    "West Virginia", "Wisconsin", "Wyoming"
]

In [19]:
from scrapy import Selector
medians =[]

for state in state_codes:
   
    populationurl= 'https://www.census.gov/quickfacts/fact/table/'+state+'/INC110221#INC110221'
   
    response = requests.get(populationurl)
    sel= Selector(text=response.text)
    medians.append(sel.css('td::text').getall()[3])

medians

['$54,943',
 '$80,287',
 '$65,913',
 '$52,123',
 '$84,097',
 '$80,184',
 '$83,572',
 '$72,724',
 '$61,777',
 '$65,030',
 '$88,005',
 '$63,377',
 '$72,563',
 '$61,944',
 '$65,429',
 '$64,521',
 '$55,454',
 '$53,571',
 '$63,182',
 '$91,431',
 '$89,026',
 '$63,202',
 '$77,706',
 '$49,111',
 '$61,043',
 '$60,560',
 '$66,644',
 '$65,686',
 '$83,449',
 '$89,703',
 '$54,020',
 '$75,157',
 '$60,516',
 '$68,131',
 '$61,938',
 '$56,956',
 '$70,084',
 '$67,587',
 '$74,489',
 '$58,234',
 '$63,920',
 '$58,516',
 '$67,321',
 '$79,133',
 '$67,674',
 '$80,615',
 '$82,400',
 '$50,884',
 '$67,080',
 '$68,002']

In [21]:
df2 = pd.DataFrame({"State":state_names,"Median Household income":medians})
df2

,State,Median Household income
0,Alabama,"$54,943"
1,Alaska,"$80,287"
2,Arizona,"$65,913"
3,Arkansas,"$52,123"
4,California,"$84,097"
5,Colorado,"$80,184"
6,Connecticut,"$83,572"
7,Delaware,"$72,724"
8,Florida,"$61,777"
9,Georgia,"$65,030"
